In [10]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

TITAN Xp
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [11]:
%load_ext autoreload
%autoreload 2

In [74]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader



import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader
from einops import rearrange, reduce, pack, unpack


In [15]:
cd motion_vqvae/

[Errno 2] No such file or directory: 'motion_vqvae/'
/coc/scratch/sanisetty3/music_motion/motion_vqvae


In [7]:
# import torch
# from torch.utils import data
# import numpy as np
# from os.path import join as pjoin
# import random
# import codecs as cs
# from tqdm import tqdm



# class VQMotionDataset(data.Dataset):
#     def __init__(self, dataset_name, window_size = 64, unit_length = 4):
#         self.window_size = window_size
#         self.unit_length = unit_length
#         self.dataset_name = dataset_name

#         if dataset_name == 't2m':
#             self.data_root = '/srv/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D'
#             self.motion_dir = pjoin(self.data_root, 'new_joint_vecs')
#             self.text_dir = pjoin(self.data_root, 'texts')
#             self.joints_num = 22
#             self.max_motion_length = 196
#             self.meta_dir = ''

#         elif dataset_name == 'kit':
#             self.data_root = './dataset/KIT-ML'
#             self.motion_dir = pjoin(self.data_root, 'new_joint_vecs')
#             self.text_dir = pjoin(self.data_root, 'texts')
#             self.joints_num = 21

#             self.max_motion_length = 196
#             self.meta_dir = 'checkpoints/kit/VQVAEV3_CB1024_CMT_H1024_NRES3/meta'
        
#         joints_num = self.joints_num

#         mean = np.load(pjoin(self.data_root, 'Mean.npy'))
#         std = np.load(pjoin(self.data_root, 'Std.npy'))

#         split_file = pjoin(self.data_root, 'val.txt')

#         self.data = []
#         self.lengths = []
#         id_list = []
#         with cs.open(split_file, 'r') as f:
#             for line in f.readlines():
#                 id_list.append(line.strip())

#         for name in tqdm(id_list):
#             try:
#                 motion = np.load(pjoin(self.motion_dir, name + '.npy'))
#                 if motion.shape[0] < self.window_size:
#                     continue
#                 self.lengths.append(motion.shape[0] - self.window_size)
#                 self.data.append(motion)
#             except:
#                 # Some motion may not exist in KIT dataset
#                 pass

            
#         self.mean = mean
#         self.std = std
#         print("Total number of motions {}".format(len(self.data)))

#     def inv_transform(self, data):
#         return data * self.std + self.mean
    
#     def compute_sampling_prob(self) : 
        
#         prob = np.array(self.lengths, dtype=np.float32)
#         prob /= np.sum(prob)
#         return prob
    
#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, item):
#         motion = self.data[item]
        
#         idx = random.randint(0, len(motion) - self.window_size)

#         motion = motion[idx:idx+self.window_size]
#         "Z Normalization"
#         motion = (motion - self.mean) / self.std

#         return motion

In [23]:
trainSet = VQMotionDataset("aist", data_root="/srv/scratch/sanisetty3/music_motion/AIST")

100%|██████████| 952/952 [00:14<00:00, 67.77it/s] 

Total number of motions 952


In [24]:
def DATALoader(
            dataset,
            batch_size,
            num_workers = 0,
            shuffle = True,
           ):
    train_loader = torch.utils.data.DataLoader(dataset,
                                                batch_size,
                                                shuffle=shuffle,
                                                #sampler=sampler,
                                                num_workers=num_workers,
                                                #collate_fn=collate_fn,
                                                drop_last = True)

    return train_loader

In [25]:
def cycle(iterable):
    while True:
        for x in iterable:
            yield x

In [53]:
train_loader = DATALoader(trainSet , 2)
train_loader_iter = cycle(train_loader)

In [54]:
gt_motion = next(train_loader_iter)

In [55]:
gt_motion.shape

torch.Size([2, 40, 263])

In [29]:
from core.models.vqvae import VQMotionModel
from core.models.loss import ReConsLoss


In [30]:
loss_fnc = ReConsLoss("l2", 22)

In [31]:
model = VQMotionModel()

VectorQuantize(
  (project_in): Identity()
  (project_out): Identity()
  (_codebook): EuclideanCodebook()
)

In [32]:
pred_motion , indices, commit_loss = model(gt_motion)

In [33]:
loss_motion = loss_fnc(pred_motion, gt_motion)


In [23]:
loss_vel = loss_fnc.forward_vel(pred_motion, gt_motion)

In [34]:
from utils.motion_process import recover_from_ric
import visualize.plot_3d_global as plot_3d
from glob import glob

In [35]:
def to_xyz(motion):
    motion_xyz = recover_from_ric(motion.cpu().float()*trainSet.std+trainSet.mean, 22)
    motion_xyz = motion_xyz.reshape(motion.shape[0],-1, 22, 3)
    return motion_xyz

            
def sample_render(motion_xyz , name , save_path):
    print(f"render start")
    
    gt_pose_vis = plot_3d.draw_to_batch(motion_xyz.numpy(),None, [os.path.join(save_path,name + ".gif")])



In [37]:
gt_motion.shape

torch.Size([8, 60, 263])

In [40]:
sample_render(to_xyz(pred_motion[0:1].detach().cpu()), "aist0_pred" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals")

render start


## Decode test

In [100]:
from core.models.vqvae import VQMotionModel

model = VQMotionModel()

In [122]:
sample_render(to_xyz(gt_motion[0:1].detach().cpu()), "gt_motion" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals/decode_test")

render start


In [124]:
pred_motion , ind1, _ = model(gt_motion)
print(pred_motion.shape, ind1.shape)

torch.Size([2, 40, 263]) torch.Size([2, 40, 8])


In [125]:
sample_render(to_xyz(pred_motion[0:1].detach().cpu()), "pred_motion" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals/decode_test")

render start


In [126]:
quantized_enc_motion, ind2, _,all_codes = model.encode(gt_motion , all_codes=True)

In [127]:
print(quantized_enc_motion.shape , ind2.shape, all_codes.shape)

torch.Size([2, 40, 128]) torch.Size([2, 40, 8]) torch.Size([8, 2, 40, 128])


In [128]:
decoded_codes , decoded_motion = model.decode_from_codebook_indices(ind2)

In [129]:
sample_render(to_xyz(decoded_motion[0:1].detach().cpu()), "dec_motion" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals/decode_test")

render start


In [ ]:
model

In [132]:
model.rq.layers[0]._codebook.embed.shape

torch.Size([1, 1024, 128])

## Music codes to motion

In [7]:
music_paths =  glob("/srv/scratch/sanisetty3/music_motion/AIST/music/*.npy" )

In [16]:
msc = np.load(music_paths[0])

In [41]:
msc2s = msc[:,:64]

In [44]:
decoded_motion_features = model.motionDecoder(torch.Tensor(msc2s.T)[None,...])

In [45]:
decoded_motion_features.shape

torch.Size([1, 64, 263])

In [46]:
sample_render(to_xyz(decoded_motion_features.detach().cpu()), "mus_pred" , "/srv/scratch/sanisetty3/music_motion/motion_vqvae/evals")

render start
